In [3]:
import pandas as pd
from tqdm import tqdm
import re

data = pd.read_csv('./process_song_data.csv')

In [25]:
def remove_the_melody(string):
    pattern = "The melody of .*? is"
    results = re.sub(pattern, '', string)
    results = re.sub(pattern.lower(), '', results)
    return results.strip()

In [26]:
remove_the_melody("The melody of 'Medley' by ABBA is catchy and upbeat, featuring a mix of different musical styles and themes that blend together seamlessly to create a memorable and enjoyable listening experience.")

'catchy and upbeat, featuring a mix of different musical styles and themes that blend together seamlessly to create a memorable and enjoyable listening experience.'

In [27]:
remove_the_melody("The song's melody is catchy and memorable, with a repetitive guitar riff and soulful vocals that create a groovy, laid-back vibe.")

"The song's melody is catchy and memorable, with a repetitive guitar riff and soulful vocals that create a groovy, laid-back vibe."

In [29]:
data = pd.read_csv('./process_song_data.csv')
data.caption = data['caption'].apply(lambda x: remove_the_melody(x))
data.to_csv('./preprocess_caption.csv')

In [30]:
data.tail()

,singer,song_name,index,emotion,caption
10283,ZZ Top,Viva Las Vegas,10304,excitement,catchy and memorable melody with a distinctive...
10284,_Weird Al_ Yankovic,Amish Paradise,10305,excitement,"catchy and upbeat, with a distinctive accordio..."
10285,_Weird Al_ Yankovic,Dare to Be Stupid,10306,excitement,"catchy and upbeat, with a memorable guitar rif..."
10286,_Weird Al_ Yankovic,Eat It,10307,excitement,"catchy and upbeat, featuring a memorable synth..."
10287,_Weird Al_ Yankovic,Headline News,10308,excitement,The catchy and upbeat melody of this song feat...


In [21]:
from datasets import load_dataset
    
dataset = load_dataset('knowledgator/events_classification_biotech') 
    
classes = [class_ for class_ in dataset['train'].features['label 1'].names if class_]
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for class_, id in class2id.items()}

/home/leesk/.local/lib/python3.8/site-packages/datasets/load.py:1454: FutureWarning: The repository for knowledgator/events_classification_biotech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/knowledgator/events_classification_biotech
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [24]:
id2class

{0: 'event organization',
 1: 'executive statement',
 2: 'regulatory approval',
 3: 'hiring',
 4: 'foundation',
 5: 'closing',
 6: 'partnerships & alliances',
 7: 'expanding industry',
 8: 'new initiatives or programs',
 9: 'm&a',
 10: 'service & product providing',
 11: 'event organisation',
 12: 'new initiatives & programs',
 13: 'subsidiary establishment',
 14: 'product launching & presentation',
 15: 'product updates',
 16: 'executive appointment',
 17: 'alliance & partnership',
 18: 'ipo exit',
 19: 'article publication',
 20: 'clinical trial sponsorship',
 21: 'company description',
 22: 'investment in public company',
 23: 'other',
 24: 'expanding geography',
 25: 'participation in an event',
 26: 'support & philanthropy',
 27: 'department establishment',
 28: 'funding round',
 29: 'patent publication'}

In [14]:
dataset[0]['label 3']

KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['test', 'train']"

In [20]:
def preprocess_function(example):
    print(example)
    text = f"{example['title']}.\n{example['content']}"
    print(example['all_labels'])
    all_labels = example['all_labels'].split(', ')
    labels = [0. for i in range(len(classes))]
    for label in all_labels:
       label_id = class2id[label]
       labels[label_id] = 1.
    print(text)
    return text
#    example = tokenizer(text, truncation=True)
#    example['labels'] = labels
#    return example

tokenized_dataset = dataset.map(preprocess_function)

Map:   0%|          | 0/2759 [00:00<?, ? examples/s]

{'title': "Sarah Polley's Book Recommendations", 'content': 'Drive Your Plow Over the Bones of The Dead\nby Olga Tokarczuk. I am an incredibly slow reader, but the tone and specificity of the world she creates in this book was something I couldnt leave behind until it was done. Also: All We Sawby Anne Michaels, Fight Nightby Miriam Toews, and The Summer Before the Darkby Doris Lessing.\nId like turned into a Netflix show:\nby Amia Srinivasan. One of the most brain-shattering books Ive ever read. Her thinking is so electrically rigorous and fearless. (I double DARE them to make this into a Netflix show!)\n...I last bought:\n. I rediscovered her poetry lately, and I feel like I dont want to read anything else for a while. She owns desire and submerged things.\n...has the greatest ending:\nby J.D. Salinger. The last page always leaves me breathless. The intimacy and truth of that final page is so arresting and almost painful to read.\nshould be on every college syllabus:\nby Anton Piatigo

AttributeError: 'list' object has no attribute 'split'

In [25]:
import torch
from transformers import AutoTokenizer, IBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("kssteven/ibert-roberta-base")
model = IBertForSequenceClassification.from_pretrained("kssteven/ibert-roberta-base", problem_type="multi_label_classification")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = IBertForSequenceClassification.from_pretrained(
    "kssteven/ibert-roberta-base", num_labels=num_labels, problem_type="multi_label_classification"
)

labels = torch.sum(
    torch.nn.functional.one_hot(predicted_class_ids[None, :].clone(), num_classes=num_labels), dim=1
).to(torch.float)
loss = model(**inputs, labels=labels).loss

tokenizer_config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of IBertForSequenceClassification were not initialized from the model checkpoint at kssteven/ibert-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'ibert.embeddings.LayerNorm.activation.act_scaling_factor', 'ibert.embeddings.LayerNorm.activation.x_max', 'ibert.embeddings.LayerNorm.activation.x_min', 'ibert.embeddings.LayerNorm.shift', 'ibert.embeddings.embeddings_act1.act_scaling_factor', 'ibert.embeddings.embeddings_act1.x_max', 'ibert.embeddings.embeddings_act1.x_min', 'ibert.embeddings.embeddings_act2.act_scaling_factor', 'ibert.embeddings.embeddings_act2.x_max', 'ibert.embeddings.embeddings_act2.x_min', 'ibert.embeddings.output_activation.act_scaling_factor', 'ibert.embeddings.output_activation.x_max', 'ibert.embeddings.output_activation.x_min', 'ibert.embeddings.position_embeddings.weight_integer', 'ibert.embeddings.position_embeddings.weight_scaling_factor', 'ibert.

In [29]:
model(**inputs).logits

tensor([[-0.1473, -0.1266]], grad_fn=<AddmmBackward0>)